# Load networks

In [ ]:
%reload_ext autoreload
%autoreload 2
import os,sys
sys.path.insert(1, os.path.join(sys.path[0], '..', 'module'))
import wiki
import numpy as np
import pandas as pd
import networkx as nx
import scipy as sp

In [ ]:
topics = ['anatomy', 'biochemistry', 'cognitive science', 'evolutionary biology',
          'genetics', 'immunology', 'molecular biology', 'chemistry', 'biophysics',
          'energy', 'optics', 'earth science', 'geology', 'meteorology',
          'philosophy of language', 'philosophy of law', 'philosophy of mind',
          'philosophy of science', 'economics', 'accounting', 'education',
          'linguistics', 'law', 'psychology', 'sociology', 'electronics',
          'software engineering', 'robotics',
          'calculus', 'geometry', 'abstract algebra',
          'Boolean algebra', 'commutative algebra', 'group theory', 'linear algebra',
          'number theory', 'dynamical systems and differential equations']

In [ ]:
path_saved = os.path.join('/','Users','harangju','Developer',
                          'data','wiki','graphs','dated')

In [ ]:
# topics = ['earth science']

In [ ]:
import wiki

networks = {}
for topic in topics:
    print(topic, end=' ')
    networks[topic] = wiki.Net(path_graph=os.path.join(path_saved, topic + '.pickle'),
                               path_barcodes=os.path.join(path_saved, topic + '.barcode'))

In [ ]:
path_null = '/Users/harangju/Developer/data/wiki/graphs/null-target/'
num_nulls = 10
null_targets = {}
for topic in topics:
    print(topic, end=' ')
    null_targets[topic] = [None for i in range(num_nulls)]
    for i in range(num_nulls):
        null_targets[topic][i] = wiki.Net(path_graph=path_null + topic + '-null-' + str(i) + '.pickle',
                                          path_barcodes=path_null + topic + '-null-' + str(i) + '.barcode')

In [ ]:
path_null = '/Users/harangju/Developer/data/wiki/graphs/null-year/'
num_nulls = 10
null_years = {}
for topic in topics:
    print(topic, end=' ')
    null_years[topic] = [None for i in range(num_nulls)]
    for i in range(num_nulls):
        null_years[topic][i] = wiki.Net(path_graph=path_null + topic + '-null-' + str(i) + '.pickle',
                                        path_barcodes=path_null + topic + '-null-' + str(i) + '.barcode')

# Awards

* [Nobel prize](https://en.wikipedia.org/wiki/List_of_Nobel_laureates)
    * [Physics](https://en.wikipedia.org/wiki/List_of_Nobel_laureates_in_Physics)
    * [Chemistry](https://en.wikipedia.org/wiki/List_of_Nobel_laureates_in_Chemistry)
    * [Physiology or Medicine](https://en.wikipedia.org/wiki/List_of_Nobel_laureates_in_Physiology_or_Medicine)
* [Fields medal](https://en.wikipedia.org/wiki/Fields_Medal)
* [Turing award](https://en.wikipedia.org/wiki/Turing_Award)
* [National Medal of Science](https://en.wikipedia.org/wiki/List_of_National_Medal_of_Science_laureates)

In [ ]:
path_base = '/Users/harangju/Developer/data/wiki/dumps/'
name_xml = 'enwiki-20190801-pages-articles-multistream.xml.bz2'
name_index = 'enwiki-20190801-pages-articles-multistream-index.txt.bz2'
path_xml = path_base + name_xml
path_index = path_base + name_index
dump = wiki.Dump(path_xml, path_index)

## Physics
[link](https://en.wikipedia.org/wiki/List_of_Nobel_laureates_in_Physics#Laureates)

In [ ]:
dump.load_page('List of Nobel laureates in Physics');
laureate_section = [s for s in dump.page.get_sections() if 'Laureates' in s[:100]][0]
laureate_section[:2000]

In [ ]:
laureate_section.filter_templates('sortname')[:3]

In [ ]:
laureate_section.filter_templates('sortname')[0].params

In [ ]:
laureates = [
    ' '.join(str(x) for x in template.params)
    for template in laureate_section.filter_templates('sortname')
]

In [ ]:
laureates[0]

In [ ]:
dump.load_page(laureates[0], filter_top=True)

In [ ]:
dump.article_links

In [ ]:
dump.page.filter_templates('Infobox')[0]

In [ ]:
dump.page.filter_templates('Infobox')[0].params

In [ ]:
type(dump.page.filter_templates('Infobox')[0].params)

In [ ]:
dump.page.filter_templates('Infobox')[0].params[14]

In [ ]:
dump.page.filter_templates('Infobox')[0].params[14].name.strip()

In [ ]:
dump.page.filter_templates('Infobox')[0].params[14].name.strip()=='known_for'

In [ ]:
known_for = [
    param for param in dump.page.filter_templates('Infobox')[0].params
    if param.name.strip()=='known_for'
][0]
known_for

In [ ]:
known_for.value

In [ ]:
known_for.value.filter_wikilinks()

In [ ]:
laureates_known_for = {}
laureates_not_found = []
for laureate in laureates:
    dump.load_page(laureate, filter_top=True)
    if not dump.page:
        laureates_not_found.append(laureate)
        continue
    infobox = dump.page.filter_templates('Infobox')
    text = None
    if len(infobox)>0:
        known_for = [
            param for param in infobox[0].params
            if param.name.strip()=='known_for'
        ]
        if len(known_for)>0:
            text = known_for[0].value
        else:
            text = dump.page
    else:
        text = dump.page
    links = text.filter_wikilinks()
    laureates_known_for[laureate] = links if len(links)>0 else text

In [ ]:
{
    laureate: known_for[:3]
    for laureate, known_for in laureates_known_for.items()
}